In [1]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,GlobalAveragePooling2D,Flatten
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
bottleneck_train=np.load('bottleneck_train_inception_v3_cancdetect')
bottleneck_valid=np.load('bottleneck_valid_inception_v3_cancdetect')
bottleneck_test=np.load('bottleneck_test_inception_v3_cancdetect')

In [3]:
bottleneck_train.shape

(2000, 6, 6, 2048)

In [4]:
trainfiles=load_files('data/train/')
train_filenames=trainfiles['filenames']
train_targets=np_utils.to_categorical(trainfiles['target'])

validfiles=load_files('data/valid/')
valid_filenames=validfiles['filenames']
valid_targets=np_utils.to_categorical(validfiles['target'])

testfiles=load_files('data/test/')
test_filenames=testfiles['filenames']
test_targets=np_utils.to_categorical(testfiles['target'])

In [5]:
model=Sequential()
model.add(GlobalAveragePooling2D(input_shape=bottleneck_train.shape[1:]))
# model.add(Dense(units=1000,activation='relu'))
model.add(Dense(units=len(trainfiles['target_names']),activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 6147      
Total params: 6,147.0
Trainable params: 6,147.0
Non-trainable params: 0.0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='rmsprop')

In [ ]:

checkpointer=ModelCheckpoint(filepath='cancdetect_transfer_inception_v3.weights.best.hdf5',verbose=1,save_best_only=True)

model.fit(bottleneck_train,train_targets,batch_size=10,epochs=100,shuffle=True,verbose=1,callbacks=[checkpointer],
             validation_data=[bottleneck_valid,valid_targets])

Train on 2000 samples, validate on 146 samples
Epoch 1/100
2000/2000 [==============================] - 20s - loss: 0.8452 - acc: 0.6625 - val_loss: 0.9160 - val_acc: 0.5822
Epoch 2/100
2000/2000 [==============================] - 1s - loss: 0.7013 - acc: 0.7245 - val_loss: 0.8634 - val_acc: 0.6301
Epoch 3/100
2000/2000 [==============================] - 1s - loss: 0.6385 - acc: 0.7285 - val_loss: 0.9305 - val_acc: 0.6027
Epoch 4/100
2000/2000 [==============================] - 1s - loss: 0.6038 - acc: 0.7450 - val_loss: 0.7759 - val_acc: 0.6438
Epoch 5/100
2000/2000 [==============================] - 1s - loss: 0.5798 - acc: 0.7560 - val_loss: 0.8834 - val_acc: 0.6301
Epoch 6/100
2000/2000 [==============================] - 1s - loss: 0.5499 - acc: 0.7820 - val_loss: 0.9648 - val_acc: 0.5959
Epoch 7/100
2000/2000 [==============================] - 1s - loss: 0.5220 - acc: 0.7810 - val_loss: 0.7795 - val_acc: 0.6849
Epoch 8/100
2000/2000 [==============================] - 1s - loss: 0.

In [ ]:
model.load_weights('cancdetect_transfer_inception_v3.weights.best.hdf5')
predictions=model.predict(bottleneck_test)

In [ ]:

ids=[filename for filename in test_filenames]
ids=[x.replace('\\','/') for x in ids]
sub_csv=[(x,y,z) for x,y,z in zip(ids,predictions[:,0],predictions[:,2])]

In [ ]:
sub_csv=[('Id','task_1','task_2')]+sub_csv

In [ ]:
import csv
with open('results_inception_v3.csv', 'w',newline='')  as myfile:
    writer = csv.writer(myfile)
#     writer.writerow('file,species')
    writer.writerows(sub_csv)